In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install git+https://github.com/SaitejaUtpala/rieoptax.git

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
  Cloning https://github.com/SaitejaUtpala/rieoptax.git to /tmp/pip-req-build-_2kp_90r
  Running command git clone --filter=blob:none --quiet https://github.com/SaitejaUtpala/rieoptax.git /tmp/pip-req-build-_2kp_90r
  Resolved https://github.com/SaitejaUtpala/rieoptax.git to commit 7e5a944ceab500b6905ca17732a6a3c295fc3afe
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for rieoptax: filename=rieoptax-0.0.1-py3-none-any.whl size=29049 sha256=517df320fc646cc0666baeeb5c7f90a9d75b7ba42d0bad241562c6741cf219a6
  Stored in directory: /tmp/pip-ephem-wheel-cache-1ahlgbb5/wheels/db/bf/d0/1494573aece1897996bcf6f222dc98d68acd429080c6f16690
  Created wheel for autodp: filename=autodp-0.2-py3-none-any.whl size=42458 sha256=32a2dae0d1d47ab7d71673c8c308164b7aa8e5065c28e217defa3efb2af7cad4
  Stored in directory: /root/.cache/pip/wheels/6a/95/85/6a1fc999268b

In [50]:
!wget https://raw.githubusercontent.com/facebookresearch/poincare-embeddings/624584cfbad684d74bf034f8dbacd515230bedc4/wordnet/mammals_filter.txt
    
    
import functools
import operator



def get_index(relations):
    all_mammals = list(sorted(set(functools.reduce(operator.iconcat, relations, []))))
    index_dict = {string: index for index, string in enumerate(all_mammals)}
    return index_dict
    

    
def get_mammal_relations():
    """Gets mammal subtree by doing transitive closure.
    Taken from https://github.com/facebookresearch/poincare-embeddings.
    """
    
    import re
    import pandas as pd
    import numpy as np
    from nltk.corpus import wordnet as wn
    from tqdm import tqdm 
    import nltk
    nltk.download('wordnet')
    nltk.download('omw-1.4')


    
    edges = set()
    for synset in tqdm(wn.all_synsets(pos='n')):
        for hyper in synset.closure(lambda s: s.hypernyms()):
            edges.add((synset.name(), hyper.name()))

        for instance in synset.instance_hyponyms():
            for hyper in instance.closure(lambda s: s.instance_hypernyms()):
                edges.add((instance.name(), hyper.name()))
                for h in hyper.closure(lambda s: s.hypernyms()):
                    edges.add((instance.name(), h.name()))

    nouns = pd.DataFrame(list(edges), columns=['id1', 'id2'])
    nouns['weight'] = 1
    mammal_set = set(nouns[nouns.id2 == 'mammal.n.01'].id1.unique())
    mammal_set.add('mammal.n.01')
    mammals = nouns[nouns.id1.isin(mammal_set) & nouns.id2.isin(mammal_set)]
    with open('mammals_filter.txt', 'r') as fin:
        filt = re.compile(f'({"|".join([l.strip() for l in fin.readlines()])})')
    filtered_mammals = mammals[~mammals.id1.str.cat(' ' + mammals.id2).str.match(filt)]

    nouns.to_csv('noun_closure.csv', index=False)
    filtered_mammals.to_csv('mammal_closure.csv', index=False)
    mammal = pd.read_csv('mammal_closure.csv')
    
    print('Total unique nodes: ', len(np.unique(list(mammal.id1.values) + list(mammal.id2.values))))
    
    mammal_relations = [[mammal.id1[i].split('.')[0], mammal.id2[i].split('.')[0]] for i in range(len(mammal))]
    mammals = sorted(np.unique([item for sublist in mammal_relations for item in sublist]))
    mammals_index = {string: index for index, string in enumerate(mammals)}
    mammal_relations_in_index = np.array([ [mammals_index[r[0]], mammals_index[r[1]]] for r in mammal_relations ])
    
    return mammal_relations, mammal_relations_in_index, mammals, mammals_index
    

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
--2023-02-23 07:11:09--  https://raw.githubusercontent.com/facebookresearch/poincare-embeddings/624584cfbad684d74bf034f8dbacd515230bedc4/wordnet/mammals_filter.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 390 [text/plain]
Saving to: ‘mammals_filter.txt.1’

mammals_filter.txt. 100%[===================>]     390  --.-KB/s    in 0s      

2023-02-23 07:11:09 (7.34 MB/s) - ‘mammals_filter.txt.1’ saved [390/390]



In [51]:
mammal_relations, mammal_relations_in_index, mammals, mammals_index = get_mammal_relations()
print(mammal_relations[0:3])
print(mammal_relations_in_index[0:3])

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
82115it [00:05, 14213.10it/s]


Total unique nodes:  1180
[['western_pipistrel', 'placental'], ['ocelot', 'carnivore'], ['mountain_sheep', 'even-toed_ungulate']]
[[1102  783]
 [ 719  198]
 [ 675  354]]


In [52]:
from dataclasses import field
from functools import partial
from typing import Any, Callable, Optional, Tuple, Union

from chex import Array
from flax import linen as nn
from flax.linen.activation import sigmoid, tanh
from flax.linen.dtypes import promote_dtype
from flax.linen.initializers import orthogonal
from flax.linen.linear import default_kernel_init
from jax import numpy as jnp
from jax import random, vmap
from chex import Array 
from jax import lax
from jax.nn.initializers import Initializer as Initializer
from jax._src import dtypes

from rieoptax.geometry.hyperbolic import PoincareBall
from rieoptax.geometry.nfold import NFoldManifold

PRNGKey = Any
KeyArray = random.KeyArray
Shape = Tuple[int, ...]
Dtype = Any
DTypeLikeFloat = Any
PrecisionLike = Union[None, str, lax.Precision, Tuple[str, str],
                      Tuple[lax.Precision, lax.Precision]]




def poincare_uniform(scale: float = 1e-2, shape=(),
            dtype: DTypeLikeFloat = jnp.float_) -> Initializer:
  """Builds an initializer that returns real (approx)uniformly-distributed random arrays
     but are constrained to poincare ball.
  Args:
    scale: optional; the upper bound of the random distribution.
    dtype: optional; the initializer's default dtype.
  Returns:
    An initializer that returns arrays whose values are uniformly distributed in
    the range ``[0, scale)``.
  """
  def init(key: KeyArray,
           shape: int,
           dtype: DTypeLikeFloat = jnp.float_) -> Array:
    
    dtype = dtypes.canonicalize_dtype(dtype)
    eps = 1e-5
    return random.uniform(key, shape, minval=-1.0+eps, maxval=1.0-eps)/shape[-1] 
   
  return init


class PoincareEmbed(nn.Module):
    """Embedding Module.
    A parameterized function from integers [0, n) to d-dimensional vectors.
    Attributes:
        num_embeddings: number of embeddings.
        features: number of feature dimensions for each embedding.
        dtype: the dtype of the embedding vectors (default: same as embedding).
        param_dtype: the dtype passed to parameter initializers (default: float32).
        embedding_init: embedding initializer.
    """
    num_embeddings: int
    features: int
    dtype: Optional[Dtype] = None
    param_dtype: Dtype = jnp.float32
    embedding_init: Callable[[PRNGKey, Shape, Dtype], Array] = poincare_uniform()
    embedding: Array = field(init=False)

    def setup(self):
        prod_manifold = NFoldManifold(PoincareBall(self.features), self.num_embeddings)
        self.embedding = self.param(f'embedding@{str(prod_manifold)}',
                                    self.embedding_init,
                                    (self.num_embeddings, self.features),
                                    self.param_dtype)

    def __call__(self, inputs: Array) -> Array: 
        """Embeds the inputs along the last dimension.
        Args:
            inputs: input data, all dimensions are considered batch dimensions.
        Returns:
            Output which is embedded input data.  The output shape follows the input,
            with an additional `features` dimension appended.
        """

        if not jnp.issubdtype(inputs.dtype, jnp.integer):
            raise ValueError('Input type must be an integer or unsigned integer.')
        embedding, = promote_dtype(self.embedding, dtype=self.dtype, inexact=False)
        return jnp.take(self.embedding, inputs, axis=0)

In [53]:
from rieoptax.geometry.hyperbolic import PoincareBall

In [54]:
model = PoincareEmbed(num_embeddings=30, features=28)

In [55]:

from jax import vmap, jit 
from functools import partial

from chex import Array


class NFoldManifold():
    def __init__(self, manifold, N):
        self.manifold = manifold
        self.N = N 
    
    
    def __str__(self):
        return f"nfold#NFoldManifold({str(self.manifold)}, {self.N})"

    @partial(jit, static_argnums=(0,))
    def inp(self, bpt: Array, tv_a: Array, tv_b: Array) -> Array:
        """Inner product between two tangent vectors at a point on manifold.

        Args:
            bpt: point on the manifold.
            tv_a: tangent vector at bpt.
            tv_b: tangent vector at bpt.

        Returns:
            returns PT_{s_pt ->e_pt}(tv).
        """
        inp = vmap(self.manifold.inp, in_axes=(0,0,0))(bpt, tv_a, tv_b)
        return inp.sum()
    
    @partial(jit, static_argnums=(0,))
    def exp(self, bpt: Array, tv: Array) -> Array:
        """Riemannian Exponential map.

        Args:
            bpt: base_point.
            tv: tangent_vec.

        Returns:
            returns Exp_{bpt}(tv).
        """
        exp = vmap(self.manifold.exp, in_axes=(0,0))(bpt, tv)
        return exp

    @partial(jit, static_argnums=(0,))
    def log(self, bpt: Array, pt: Array) -> Array:
        """Riemannian Logarithm map.

        Args:
            bpt: base_point.
            pt: tangent_vec.

        Returns:
            returns Log_{bpt}(pt).
        """
        log = vmap(self.manifold.log, in_axes=(0,0))(bpt, pt)
        return log
    
    @partial(jit, static_argnums=(0,))
    def egrad_to_rgrad(self, bpt: Array, egrad: Array) -> Array:
        """Euclidean gradient to Riemannian Gradient Convertor.

        Args:
            bpt: base_point.
            egrad: tangent_vec.

        Returns:
            returns Log_{bpt}(pt).
        """
        rgrad = vmap(self.manifold.egrad_to_rgrad, in_axes=(0,0))(bpt, egrad)
        return rgrad

    @partial(jit, static_argnums=(0,))
    def dist(self, pt_a: Array, pt_b: Array) -> float:
        """Distance between two points on the manifold induced by Riemannian metric.

        Args:
            pt_a: point on the manifold.
            pt_b: point on the manifold.

        Returns:
            returns distance between pt_a, pt_b.
        """
        dist = vmap(self.manifold.dist, in_axes=(0,0))(pt_a, pt_a)
        return dist.sum()

    @partial(jit, static_argnums=(0,))
    def ptrans(self, s_pt: Array, e_pt: Array, tv: Array) -> Array:
        """Parallel Transport.

        Args:
            s_pt: start point.
            e_pt: end point.
            tv: tangent vector at start point.

        Returns:
            returns PT_{s_pt ->e_pt}(tv).
        """
        ptrans = vmap(self.manifold.ptrans, in_axes=(0,0,0))(s_pt, e_pt, tv)
        return ptrans 


In [56]:
from flax.struct import PyTreeNode, field
from flax.core import FrozenDict
from rieoptax.optimizers.update import apply_updates
from typing import Optional, Any, Callable
from rieoptax.optimizers.base import (
    RiemannianGradientTransformation,
    get_manifold_dict,
    rgrad_from_egrad,
)

OptState = Any


class RtxTrainState(PyTreeNode):
    step: int
    apply_fn: Callable = field(pytree_node=False)
    params: FrozenDict[str, Any]
    rtx: RiemannianGradientTransformation = field(pytree_node=False)
    opt_state: OptState
    manifold_dict: FrozenDict[str, Any] = field(pytree_node=False)
    update_fn: str = "exp"
    

    def apply_gradients(self, *, egrads, **kwargs):
        rgrads = rgrad_from_egrad(self.params, egrads, self.manifold_dict)
        updates, new_opt_state = self.rtx.update(rgrads, self.opt_state, self.params) 
        new_params = apply_updates(
            self.params, updates, self.manifold_dict, self.update_fn
        )
        return self.replace(
            step=self.step + 1,
            params=new_params,
            opt_state=new_opt_state,
            **kwargs,
        )

    @classmethod
    def create(cls, *, apply_fn, params, rtx, **kwargs):
        """Creates a new instance with `step=0` and initialized `opt_state`."""
        opt_state = rtx.init(params)
        manifold_dict = get_manifold_dict(params)
        
        return cls(
            step=0,
            apply_fn=apply_fn,
            params=params,
            rtx=rtx,
            opt_state=opt_state,
            manifold_dict=manifold_dict,
            **kwargs,
        )


In [57]:
#from rieoptax.optimizers.rtx_train_state import RtxTrainState
from rieoptax.optimizers.first_order import rsgd 

def create_train_state(module, rng, learning_rate):
    """Creates an initial `TrainState`."""
    params = module.init(rng, jnp.array([0, 0]))
    rtx = rsgd(learning_rate)
    return RtxTrainState.create(apply_fn=module.apply, params=params, rtx=rtx)

In [58]:
import jax

@jax.jit
def train_step(state, batch):
    
    def loss_fn(params):
        
        @partial(jax.vmap, in_axes=(0,0,0))
        def _loss_fn(u,v,neg):
            u_embed = state.apply_fn(params, u)
            v_embed = state.apply_fn(params, v)
            neg_embed =  state.apply_fn(params, neg)
            
            u_neg = jnp.linalg.norm(u_embed[None]-neg_embed, dim=-1)
            nomin = -1*jnp.linalg.norm(u_embed-v_embed)
            denom = jax.nn.logsumexp(-1*u_neg)
            return nomin/denom
    
        loss = _loss_fn(batch).mean()
        return loss
    
    egrad_fn = jax.grad(loss_fn)
    egrads = egrad_fn(state.params)
    state = state.apply_gradients(egrads=egrads)
    return state


@jax.jit
def eval_step(state, batch):
    image, label = batch
    logits = state.apply_fn({'params': state.params}, image)
    return compute_metrics(logits=logits, labels=label)

In [67]:
import numpy as np 
def data_generator(
    mammal_relations_in_index,
    n_mammals,
    batch_size=10,
    n_samples=10,
    is_valid=False,
    key=None,
):
    def negative_sample(relations, mammals, n_samples):
        neg_samples_list = []
        for relation in relations:
            u = relation[0]
            positives = [x[1] for x in relations if x[0] == u]
            negatives = np.array([x for x in mammals if x not in positives])
            random_ix = np.random.permutation(len(negatives))[:n_samples]

            neg_samples = [[u, x] for x in negatives[random_ix]]
            neg_samples.append([u, u])
            neg_samples_list.append(neg_samples)
        
        return np.array(neg_samples_list)

    
    
    mammals_no_list = np.arange(0, n_mammals)
    num_batches = int(np.ceil(len(mammal_relations_in_index) / batch_size))
    indices = np.arange(len(mammal_relations_in_index))

    if not is_valid:
        if key is None:
            raise ValueError("A PRNG key is required if `aug` is set to True")
        else:
            np.random.shuffle(indices)

    for batch in range(num_batches):
        curr_idx = indices[batch * batch_size : (batch + 1) * batch_size]
        sampled_mammals_relations = mammal_relations_in_index[curr_idx]
        neg_samples = negative_sample(
            sampled_mammals_relations, mammals_no_list, n_samples
        )

        yield sampled_mammals_relations, neg_samples


In [71]:
data_loader = data_generator(mammal_relations_in_index,len(mammals), key=key)
sampled_relations, neg_samples = next(data_loader)
print("Batch of images is of shape: ", sampled_relations.shape)
print("Batch of labels is of shape: ", neg_samples.shape)

Batch of images is of shape:  (10, 2)
Batch of labels is of shape:  (10, 11, 2)
